In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin


In [2]:
import requests
from bs4 import BeautifulSoup

# Peta bulan ke format angka
bulan_ke_angka = {
    'Januari': '01',
    'Februari': '02',
    'Maret': '03',
    'April': '04',
    'Mei': '05',
    'Juni': '06',
    'Juli': '07',
    'Agustus': '08',
    'September': '09',
    'Oktober': '10',
    'November': '11',
    'Desember': '12'
}

def format_tanggal(tanggal):
    bagian = tanggal.split(' ')
    hari = bagian[0]
    bulan = bagian[1]
    tahun = bagian[2]

    bulan_angka = bulan_ke_angka[bulan]
    return f"{hari.zfill(2)}-{bulan_angka}-{tahun}"

def scrape_website(base_url, max_pages):
    scraped_data = []

    for page in range(1, max_pages + 1):
        # Buat URL untuk setiap halaman
        page_url = f"{base_url}&page={page}"
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Ambil semua artikel dari halaman tersebut
        articles = soup.find('div', class_="latest__wrap")
        articles = articles.find_all('div', class_="latest__item")

        for article in articles:
            title_element = article.find('h2').find('a')
            title = title_element.text.strip()

            if 'bpjs' in title.lower():
                # Ambil link artikel
                link_element = title_element
                link = link_element.attrs['href']

                # Ambil tanggal artikel
                date_element_1 = article.find('div', 'latest__right')
                date_element_2 = date_element_1.find('date', 'latest__date')
                date_full = date_element_2.text.strip()

                # Pisahkan tanggal dari waktu
                date = date_full.split('|')[0].strip()
                date = date.split(', ')[1]

                # Format tanggal menjadi DD-MM-YYYY
                formatted_date = format_tanggal(date)

                if not any(d['Link'] == link for d in scraped_data):
                    scraped_data.append({
                        'Link': link,
                        'Tanggal': formatted_date
                    })

    return scraped_data

# Contoh penggunaan
base_url = "https://news.indozone.id/search?q=bpjs&sort=latest"
max_pages = 19  # Scrap hingga halaman ke-19
data = scrape_website(base_url, max_pages)

# Print hasilnya
for item in data:
    print(item)


{'Link': 'https://news.indozone.id/news/915064988/lansia-di-jaksel-rugi-rp1-m-usai-jadi-korban-penipuan-pelaku-pura-pura-jadi-petugas-bpjs-dan-polisi', 'Tanggal': '07-09-2024'}
{'Link': 'https://news.indozone.id/indonesia-hari-ini/914783072/respon-abdi-dalem-keraton-jogja-soal-wacana-bpjs-ketenagakerjaan', 'Tanggal': '21-06-2024'}
{'Link': 'https://news.indozone.id/news/914774986/alasan-dprd-kota-jogja-usulkan-abdi-dalem-dapat-bpjs-ketenagakerjaan', 'Tanggal': '19-06-2024'}
{'Link': 'https://news.indozone.id/news/914726440/iuran-bpjs-kesehatan-bisa-dicairkan-jika-peserta-tak-pernah-sakit-ini-penjelasannya', 'Tanggal': '05-06-2024'}
{'Link': 'https://news.indozone.id/news/914724357/3-cara-cetak-kartu-bpjs-kesehatan-secara-online-dengan-mudah-bisa-via-hp', 'Tanggal': '04-06-2024'}
{'Link': 'https://news.indozone.id/indonesia-hari-ini/914723055/catat-mulai-juli-2024-bpjs-jadi-syarat-bikin-dan-perpanjang-sim', 'Tanggal': '04-06-2024'}
{'Link': 'https://news.indozone.id/news/914692551/lupa-

In [3]:
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

In [5]:
save_to_csv(data, 'link_indozone.csv')